In [1]:
import datetime, urllib, re, math, json, requests,io
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
from scipy import optimize
from sklearn.linear_model import LinearRegression
from IPython.display import display, HTML



try:
    from packaging import version
except ImportError:
    !pip install packaging

if( version.parse(pd.__version__) < version.parse("0.23.4")):
    print("update pandas")
    !pip install pandas --upgrade #--ignore-installed
    print(version.parse(pd.__version__))

In [2]:
#** pandas
# change max number of rows to show
pd.set_option('display.max_rows', 300)

#** plotters formats

import matplotlib.ticker as mticker
import matplotlib.dates as mdates
from cycler import cycler
#* dpi size
plt.rcParams['figure.dpi'] = 200
myFmt = mdates.DateFormatter('%m/%d')
myLocator = mticker.MultipleLocator(7)

default_cycler = (cycler(marker=['.','*','+','s','x']) *
                  cycler(color=['b','g','k','m','y','c','r']) *
                  cycler(linestyle=['-']))

plt.rc('axes', prop_cycle=default_cycler)

#** panda data frame styles
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)


In [209]:
df = pd.read_csv("https://coronadatascraper.com/timeseries-tidy.csv")
df.fillna("NA",inplace=True);
dataTCS =df.set_index(["city","county","state","country","type"])
dataTCS["date"] = [datetime.datetime.strptime(ts,"%Y-%m-%d").date() for ts in dataTCS["date"]]


In [237]:
from IPython.html.widgets import *

def foo(a,b): return a+b

interact(foo,a=(0,10,.5),b=(0,15,.5))

interactive(children=(FloatSlider(value=5.0, description='a', max=10.0, step=0.5), FloatSlider(value=7.0, desc…

<function __main__.foo(a, b)>

In [ ]:
minCount     = 500
minCountKpi  = "deaths" # 'active', 'cases', 'deaths', 'growthFactor', 'recovered', 'tested'
equalize     = True
equalizeTrg  = "USA"
equalizeCount= 400

plotKpi      = "deaths"
logyPlot     = True

idx = pd.IndexSlice


fig,ax = plt.subplots(1,1)

if(equalize):
    g = dataTCS.loc[idx["NA","NA","NA",equalizeTrg,minCountKpi]]
    gdts = g[g["value"]>100]["date"].values[0]

    print(gdts)

# temp commented out    
for ctr in dataTCS.index.levels[3]:

    if dataTCS.loc[idx["NA","NA","NA",ctr,minCountKpi]].value[-1]< minCount: continue
#     
    g = dataTCS.loc[idx["NA","NA","NA",ctr,plotKpi],["date","value"]]
    dts = g[g["value"]>equalizeCount]["date"].values[0]
    print("%s --> %d" %(ctr,(dts-gdts).days))
    data= g[g["date"]>=dts]
    ts = [(gdts + datetime.timedelta(i)) for i in range(0,sp.size(data["value"]))] 
    ax.plot_date(ts,data["value"].values,label=ctr)    
ax.xaxis.set_major_locator(myLocator)
ax.grid()
ax.xaxis.set_major_formatter(myFmt)
ax.legend()
ax.set_title("new cases vs time");
#print(dataTCS.loc[idx["NA","NA","NA",ctr,plotKpi],"value"].values)

if(logyPlot) : plt.yscale('log')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: PerformanceWarning: indexing past lexsort depth may impact performance.
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: PerformanceWarning: indexing past lexsort depth may impact performance.


2020-03-17
CHN --> -43
DEU --> 11
ESP --> 0
FRA --> 3
GBR --> 7
IRN --> -5
ITA --> -8
NLD --> 9
USA --> 6
